# Reader

We've already explored the *reader* model in the first few sections of this chapter, where we loaded a pretrained model using the `transformers` library.

Now, we'll do the same thing but via Haystack - which uses the same `transformers` library in the background (so we can use the same pretrained model names).

To initialize our reader via Haystack all we need is:

In [1]:
from haystack.nodes import FARMReader

2022-11-14 21:17:59.860964: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-14 21:17:59.976428: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-14 21:18:00.290027: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-11-14 21:18:00.290099: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

In [2]:
reader = FARMReader(model_name_or_path='deepset/bert-base-cased-squad2', use_gpu=True)

Now we have our reader model initialized, let's load up our FAISS index and DPR just like before.

In [3]:
# from haystack.document_store.faiss import FAISSDocumentStore
# from haystack.retriever.dense import DensePassageRetriever
from haystack.document_stores.faiss import FAISSDocumentStore
from haystack.nodes.retriever.dense import DensePassageRetriever

path = '../../models/faiss'

# load FAISS from file (the squad validation set index)
# document_store = FAISSDocumentStore.load(f'{path}/squad_dev.faiss', f'sqlite:///{path}/squad_dev.db')
# document_store = FAISSDocumentStore(index_path=f'{path}/squad_dev.faiss', f'sqlite:///{path}/squad_dev.db')
document_store = FAISSDocumentStore.load(index_path=f'{path}/squad_dev.faiss', config_path=f'{path}/squad_dev.json')
# initialize DPR model
retriever = DensePassageRetriever(
    document_store=document_store,
    query_embedding_model='facebook/dpr-question_encoder-single-nq-base',
    passage_embedding_model='facebook/dpr-ctx_encoder-single-nq-base',
    use_gpu=True,
    embed_title=True
)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRContextEncoderTokenizerFast'.


Haystack comes with a very convenient `ExtractiveQAPipeline` class which allows us to pass our `reader` and `retriever` to build an easy-to-use *extractive* Q&A pipeline:

In [6]:
# from haystack.pipeline import ExtractiveQAPipeline
from haystack.pipelines import ExtractiveQAPipeline

pipeline = ExtractiveQAPipeline(reader=reader, retriever=retriever)

We can ask questions using the `run` method, alongside the `query` parameter:

In [18]:
pipeline.run(query='Will there ever be born a boy who can swim as fast as a shark?')['answers']

Inferencing Samples: 100%|██████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  4.20 Batches/s]


[<Answer {'answer': '1653 the poet Zygmunt Laukowski', 'type': 'extractive', 'score': 0.03928980231285095, 'context': ' use of a crude form of a sea monster with a female upper body and holding a sword in its claws. In 1653 the poet Zygmunt Laukowski asks the question:', 'offsets_in_document': [{'start': 521, 'end': 552}], 'offsets_in_context': [{'start': 100, 'end': 131}], 'document_id': 'bf117db136b152b4b30ad6fb942b6aae', 'meta': {'vector_id': '879'}}>,
 <Answer {'answer': 'juveniles are capable of reproduction before reaching the adult size and shape.', 'type': 'extractive', 'score': 0.0005937825189903378, 'context': 'ult form. In at least some species, juveniles are capable of reproduction before reaching the adult size and shape. The combination of hermaphroditism', 'offsets_in_document': [{'start': 912, 'end': 991}], 'offsets_in_context': [{'start': 36, 'end': 115}], 'document_id': 'e88147b5aea54c44d788c726b5555167', 'meta': {'vector_id': '1083'}}>,
 <Answer {'answer': '2009', 't

Here we're returning a lot of different answers, which is not really necessary. We can limit the number of answers we return using two parameters, `top_k_retriever` and `top_k_reader`. Both of these limit the number of items being returned from the `retriever` and `reader` models respectively.

In [ ]:
# Broken in latest version of haystack, couldn't find fix
# pipeline.run(query='What does theoretical computer science cover?',
#              top_k_retriever=3)

TypeError: run() got an unexpected keyword argument 'top_k_retriever'

And the `reader`:

In [13]:
# Broken in latest version of haystack, couldn't find fix
# pipeline.run(query='What does theoretical computer science cover?',
#              top_k_reader=3)

TypeError: run() got an unexpected keyword argument 'top_k_reader'

In [19]:
from haystack.document_stores.elasticsearch import ElasticsearchDocumentStore

def reading(file_name = 'credentials.txt'):
    s = open(file_name, 'r').read()
    dict = eval(s)
    return(dict)

credential_dict = reading()

document_store = ElasticsearchDocumentStore(
    host='localhost', 
    scheme='https', 
    username=credential_dict['username'], 
    password=credential_dict['pwd'], 
    ca_certs=credential_dict['ca_certs'], index='squad_docs')

In [20]:
# initialize DPR model
retriever = DensePassageRetriever(
    document_store=document_store,
    query_embedding_model='facebook/dpr-question_encoder-single-nq-base',
    passage_embedding_model='facebook/dpr-ctx_encoder-single-nq-base',
    use_gpu=True,
    embed_title=True
)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRContextEncoderTokenizerFast'.


In [21]:
document_store.update_embeddings(retriever=retriever)

Updating embeddings:   0%|          | 0/1204 [00:00<?, ? Docs/s]

Create embeddings:   0%|          | 0/1216 [00:00<?, ? Docs/s]

In [22]:
pipeline = ExtractiveQAPipeline(reader=reader, retriever=retriever)

In [23]:
pipeline.run(query='Will there ever be born a boy who can swim as fast as a shark?')['answers']

Inferencing Samples: 100%|██████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  3.60 Batches/s]


[<Answer {'answer': '1653 the poet Zygmunt Laukowski', 'type': 'extractive', 'score': 0.03928980231285095, 'context': ' use of a crude form of a sea monster with a female upper body and holding a sword in its claws. In 1653 the poet Zygmunt Laukowski asks the question:', 'offsets_in_document': [{'start': 521, 'end': 552}], 'offsets_in_context': [{'start': 100, 'end': 131}], 'document_id': 'bf117db136b152b4b30ad6fb942b6aae', 'meta': {}}>,
 <Answer {'answer': 'juveniles are capable of reproduction before reaching the adult size and shape.', 'type': 'extractive', 'score': 0.0005937825189903378, 'context': 'ult form. In at least some species, juveniles are capable of reproduction before reaching the adult size and shape. The combination of hermaphroditism', 'offsets_in_document': [{'start': 912, 'end': 991}], 'offsets_in_context': [{'start': 36, 'end': 115}], 'document_id': 'e88147b5aea54c44d788c726b5555167', 'meta': {}}>,
 <Answer {'answer': '2009', 'type': 'extractive', 'score': 0.000492